In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import csv 
import re
import matplotlib.pyplot as plt
from IPython.display import Image

In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
df = pd.read_csv('fiuba_6_avisos_detalle_concat.csv', delimiter=',')
tuples = [tuple(x) for x in df.values]
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,1,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,1,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [4]:
for x in tuples:
    x = tuple(x)

In [5]:
avisos = sc.parallelize(tuples)
avisos.take(1)

[(0,
  1111556097,
  1,
  'Responsable de Marketing-Producto Div. Construcción -GBA Sur ',
  '<p>Buscamos un <strong>Responsable\xa0de Producto</strong> para incorporarse al departamento de Marketing de la Cía., en nuestras oficinas centrales ubicadas en Planta Ezeiza (C.Spegazzini).</p><p>\xa0</p><p>Será responsable de una cartera de productos de la línea Construcción.</p><p>Sus tareas incluyen realizar análisis de costos, competencia, acciones promocionales, estrategias de comunicación, políticas de precios, desarrollo de nuevos productos y reportes de gestión, contacto con clientes y solución de necesidades, entre otras.</p><p>Horario de trabajo: de Lunes a Viernes de 8 a 17 hs.</p><p>\xa0</p><p>Buscamos un profesional graduado\xa0de las carreras de Arquitectura, Ing. Civil, Marketing, Comercialización, Adm. de empresas o similares, con experiencia afín desarrollada en empresas del rubro (construcción, químicas, etc).</p><p>Se valorarán los conocimientos en idioma inglés,\xa0sistema

In [6]:
avisos = avisos.map(lambda x: (x[0],x[1],x[2],str.upper(re.sub("<.*?>", " ", x[3])),str.upper(re.sub("<.*?>", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10],x[11])).map(lambda x: (x[0],x[1],x[2],str(re.sub("\xa0", " ", x[3])),str(re.sub("\xa0", " ", x[4])),x[5],x[6],x[7],x[8],x[9],x[10],x[11]))
avisos.take(1)

[(0,
  1111556097,
  1,
  'RESPONSABLE DE MARKETING-PRODUCTO DIV. CONSTRUCCIÓN -GBA SUR ',
  ' BUSCAMOS UN  RESPONSABLE DE PRODUCTO  PARA INCORPORARSE AL DEPARTAMENTO DE MARKETING DE LA CÍA., EN NUESTRAS OFICINAS CENTRALES UBICADAS EN PLANTA EZEIZA (C.SPEGAZZINI).     SERÁ RESPONSABLE DE UNA CARTERA DE PRODUCTOS DE LA LÍNEA CONSTRUCCIÓN.  SUS TAREAS INCLUYEN REALIZAR ANÁLISIS DE COSTOS, COMPETENCIA, ACCIONES PROMOCIONALES, ESTRATEGIAS DE COMUNICACIÓN, POLÍTICAS DE PRECIOS, DESARROLLO DE NUEVOS PRODUCTOS Y REPORTES DE GESTIÓN, CONTACTO CON CLIENTES Y SOLUCIÓN DE NECESIDADES, ENTRE OTRAS.  HORARIO DE TRABAJO: DE LUNES A VIERNES DE 8 A 17 HS.     BUSCAMOS UN PROFESIONAL GRADUADO DE LAS CARRERAS DE ARQUITECTURA, ING. CIVIL, MARKETING, COMERCIALIZACIÓN, ADM. DE EMPRESAS O SIMILARES, CON EXPERIENCIA AFÍN DESARROLLADA EN EMPRESAS DEL RUBRO (CONSTRUCCIÓN, QUÍMICAS, ETC).  SE VALORARÁN LOS CONOCIMIENTOS EN IDIOMA INGLÉS, SISTEMA SAP Y EXCEL.     OFRECEMOS LA POSIBILIDAD DE INCORPORARSE A UNA EM

In [7]:
import re, string, unicodedata
def removePunctuation(text):
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', '', text)   
    return text

In [8]:
def palabrasLargas(texto):
    lista = texto.split(' ') 
    lista = [x for x in lista if not len(x)>17]
    return " ".join(lista)

In [9]:
avisos.take(1)

[(0,
  1111556097,
  1,
  'RESPONSABLE DE MARKETING-PRODUCTO DIV. CONSTRUCCIÓN -GBA SUR ',
  ' BUSCAMOS UN  RESPONSABLE DE PRODUCTO  PARA INCORPORARSE AL DEPARTAMENTO DE MARKETING DE LA CÍA., EN NUESTRAS OFICINAS CENTRALES UBICADAS EN PLANTA EZEIZA (C.SPEGAZZINI).     SERÁ RESPONSABLE DE UNA CARTERA DE PRODUCTOS DE LA LÍNEA CONSTRUCCIÓN.  SUS TAREAS INCLUYEN REALIZAR ANÁLISIS DE COSTOS, COMPETENCIA, ACCIONES PROMOCIONALES, ESTRATEGIAS DE COMUNICACIÓN, POLÍTICAS DE PRECIOS, DESARROLLO DE NUEVOS PRODUCTOS Y REPORTES DE GESTIÓN, CONTACTO CON CLIENTES Y SOLUCIÓN DE NECESIDADES, ENTRE OTRAS.  HORARIO DE TRABAJO: DE LUNES A VIERNES DE 8 A 17 HS.     BUSCAMOS UN PROFESIONAL GRADUADO DE LAS CARRERAS DE ARQUITECTURA, ING. CIVIL, MARKETING, COMERCIALIZACIÓN, ADM. DE EMPRESAS O SIMILARES, CON EXPERIENCIA AFÍN DESARROLLADA EN EMPRESAS DEL RUBRO (CONSTRUCCIÓN, QUÍMICAS, ETC).  SE VALORARÁN LOS CONOCIMIENTOS EN IDIOMA INGLÉS, SISTEMA SAP Y EXCEL.     OFRECEMOS LA POSIBILIDAD DE INCORPORARSE A UNA EM

In [10]:
avisos = avisos.map((lambda x: (x[0],x[1],x[2],removePunctuation(x[3]),removePunctuation(x[4]),x[5],x[6],x[7],x[8],x[9],x[10],x[11]))).map((lambda x: (x[0],x[1],x[2],palabrasLargas(x[3]),x[4],x[5],x[6],x[7],x[8], x[9],x[10],x[11]))).collect()

In [11]:
avisos = pd.DataFrame.from_records(avisos).drop([0], axis=1)

In [12]:
for x in avisos.columns:  
    avisos = avisos.rename(columns = {x:df.columns[x-1]})
avisos.head()
avisos = avisos.drop(['idpais','nombre_zona','mapacalle', 'ciudad'], axis=1)
avisos.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,responsable de marketingproducto div construcc...,buscamos un responsable de producto para inc...,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,asesor de servicios postventa chevrolet,para nuestro cliente concesionario chevrolet u...,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,analistas de impuestos sr ssr y jr,importante estudio de auditoria y consultoria ...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,software engineer,chevron is a multinational company in the ener...,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,business developer manager,importante empresa de energias renovables se e...,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


(25288, 7)

In [14]:
cluster = pd.read_csv('idaviso-cluster.csv')

In [15]:
avisos = avisos.merge(cluster,how='inner',on='idaviso')

In [16]:
avisos.sample()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Unnamed: 0,cluster
23035,1112507461,coordinacion de trabajo social para empresa de...,en diaverum nuestra mision es mejorar la calid...,Full-time,Jefe / Supervisor / Responsable,Medicina,DIAVERUM ARGENTINA S.A.,23035,11


In [19]:
avisos.drop(['Unnamed: 0'],axis=1,inplace=True)

In [20]:
avisos.shape

(25288, 8)

In [21]:
avisos.to_csv("fiuba_6_avisos_detalle_limpio.csv")